In [1]:
import numpy as np
import pandas as pd
from BrMiner.io import load_mzml

In [2]:
ms1, ms2 = load_mzml("Br_standards_8Mix.mzML.gz")

In [3]:
MIN_PROB_FOR_MS1 = 0.01
MS2_DIFF_THRESHOLD = 0.01
N_BRS = list(range(3, 16))
MASS_ACC = 5e-6
PROB_LIMIT= 0.8

In [4]:
from BrMiner.algorithm import MSPatternFinder
from BrMiner.elements import EDB
from BrMiner.utils import TqdmParallel
from collections import defaultdict, namedtuple
import joblib

C_ele = EDB['C']
C_diff = C_ele.fuzzy_find(13).mass - C_ele.fuzzy_find(12).mass
Entry = namedtuple('Entry',['MS1_IDX', 'RT', 'MZ', 'N_Br', 'Prob', 'MZ_Sel', 'INT_Sel'])
finders = [MSPatternFinder(elements={'Br': n_br}, mass_acc=5e-6, top_n=(6 if n_br % 2 == 0 else 5), intensity_lb=1e4) for n_br in N_BRS]
#resutls = {}

#for ms1_spec in tqdm.tqdm(obj.ms1[['RT', 'MZ', 'INT']].itertuples(), total=obj.ms1.shape[0]):
@joblib.delayed
def compute(ms1_spec):
    global finders, PROB_LIMIT, C_diff
    dic = defaultdict(list)
    for f in finders:
        for pattern in f.match(ms1_spec.MZ, ms1_spec.INT):
            if pattern.certainty >= PROB_LIMIT:
                dic[pattern.mz].append((f.elements['Br'], pattern))
    resutls = []
    for k in sorted(dic.keys()):
        v = dic[k]
        n_br, pattern = max(v, key=lambda x: x[1].certainty)
        e = Entry(ms1_spec.Index, ms1_spec.RT, pattern.mz, n_br, pattern.certainty, pattern.mz_sel, pattern.intensity_sel)
        iso_min = (1-MASS_ACC)*(e.MZ/(1+MASS_ACC) - C_diff)
        iso_max = (1+MASS_ACC)*(e.MZ/(1-MASS_ACC) - C_diff)
        for p in resutls:
            if iso_min <= p.MZ and p.MZ <= iso_max:
                break
        else:
            resutls.append(e)
    return resutls
resutls = TqdmParallel(-1, verbose=0)([compute(ms1_spec) for ms1_spec in ms1[['RT', 'MZ', 'INT']].itertuples()])

100%|██████████| 2740/2740 [00:14<00:00, 191.30it/s]


In [5]:
import functools, operator
resutls_all = pd.DataFrame(functools.reduce(operator.add, resutls))

In [6]:
def simple_ms1(peaks: pd.DataFrame, rt_diff=30, mz_diff=0.01):
    """Group peaks"""
    from scipy.sparse import lil_matrix
    from scipy.sparse.csgraph import connected_components

    graph = lil_matrix((peaks.shape[0], peaks.shape[0]))
    rt = peaks["RT"]
    mz = peaks["MZ"]
    for i in range(peaks.shape[0]):
        dist = np.logical_and(
            np.abs(rt[i] - rt) < rt_diff, np.abs(mz[i] - mz) < mz_diff
        )
        graph[i, :] = dist
    _, labels = connected_components(graph, directed=False)
    new_peaks = peaks.assign(Label=labels)
    ulables = np.unique(labels)

    final_results = []
    for lbl in ulables:
        sel = lbl == labels
        group_peaks = peaks.iloc[sel]

        peaks_ints = group_peaks["INT_Sel"].apply(lambda x: x[x.size // 2])
        representer = group_peaks.loc[peaks_ints.idxmax()].copy()
        representer["INT"] = peaks_ints.max()

        representer["All_RT"] = group_peaks["RT"].to_numpy()
        representer["All_MS1_IDX"] = group_peaks["MS1_IDX"].to_numpy()
        final_results.append(representer)
    final_results = pd.DataFrame(final_results)
    return final_results, new_peaks
final_results, new_peaks = simple_ms1(resutls_all, rt_diff=30)

In [7]:
from BrMiner.predict import predict_formula
from BrMiner.utils import TqdmParallel
import joblib

@joblib.delayed
def f(mass, n_br, charge):
    n_Br81 = n_br // 2
    n_Br79 = n_br  - n_Br81
    min_DoU = 0
    formulas = predict_formula(mass,
                               mass_acc=5e-6,
                               charge=charge,
                               lim_C=range(100),
                               lim_H=range(200),
                               lim_O=range(0, 40),
                               lim_DoU=range(min_DoU, 51),
                               lim_N=range(4),
                               lim_P=[0],
                               lim_Br79=[n_Br79],
                               lim_Br81=[n_Br81]
                               )
    return formulas
tasks = []
for s in final_results[['MZ', 'N_Br']].itertuples():
    tasks.append(f(s.MZ, s.N_Br, charge=-1))
formula_pos = TqdmParallel(-1)(tasks)

  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [00:07<00:00,  3.56it/s]


In [8]:
def expToStr(flist):
    if flist:
        return " | ".join(fm.empirical_formula() for fm in flist)
    else:
        return ""
results = final_results.assign(formula = list(map(expToStr, formula_pos)))

In [9]:
results[["RT","MZ","INT","N_Br","Prob",'formula']].to_excel('formulas.xlsx')